In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

In [2]:
model = tf.keras.models.load_model('combine_prediction_model.keras')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1856      
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 12417 (48.50 KB)
Trainable params: 12417 (48.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [3]:
player_values = input("Please input combine stats in this format\n(Year, POS, Height (in), Weight (lbs), 40 Yard, Bench Press, Vertical Leap (in), Broad Jump (in), Shuttle, 3 Cone)\nEnsure no spaces in between values only seperating values with a comma:\n")
player_values_list = player_values.split(",")

player_Year, player_POS, player_Height, player_Weight, player_40Yard, player_BenchPress, player_Vert, player_Broad, player_Shuttle, player_3Cone = player_values_list

In [4]:
print(player_Year)

2011


In [5]:
#player_Year = 2022
#player_POS = 'CB'
#player_Height = 74.75
#player_Weight = 190
#player_40Yard = 4.41
#player_BenchPress = 21
#player_Vert = 32.5
#player_Broad = 118
#player_Shuttle = 4.51
#player_3Cone = 7.31

In [6]:
holder_df = pd.DataFrame(columns=['Year', 'POS', "Height (in)", "Weight (lbs)", "40 Yard", "Bench Press", "Vert Leap (in)", "Broad Jump (in)", "Shuttle", "3Cone"])
new_entry = pd.DataFrame({'Year': [float(player_Year)], 'POS': [player_POS], "Height (in)": [float(player_Height)], "Weight (lbs)": [float(player_Weight)], "40 Yard": [float(player_40Yard)], "Bench Press": [float(player_BenchPress)], "Vert Leap (in)": [float(player_Vert)], "Broad Jump (in)": [float(player_Broad)], "Shuttle": [float(player_Shuttle)], "3Cone": [float(player_3Cone)]})
holder_df = pd.concat([holder_df,new_entry], ignore_index=True)

In [7]:
sampled_df = pd.read_csv('sampled_data.csv')
sampled_df = sampled_df.drop(columns=["Pro Bowl"])
pred_df = pd.concat([sampled_df, holder_df], ignore_index=True)

In [8]:
print(pred_df.head())

     Year POS  Height (in)  Weight (lbs)  40 Yard  Bench Press  \
0  2009.0   K        72.88         220.0     4.78         22.0   
1  1989.0  DE        75.50         256.0     4.85         20.0   
2  2020.0  DE        76.88         264.0     4.74         20.0   
3  2016.0  WR        68.13         185.0     4.29         13.0   
4  1994.0   C        74.60         270.0     4.90         19.0   

   Vert Leap (in)  Broad Jump (in)  Shuttle  3Cone  
0            33.0            113.0     4.45   7.31  
1            31.4            110.0     4.44   7.31  
2            33.5            116.0     4.46   7.31  
3            40.5            129.0     4.06   6.53  
4            30.7            111.0     4.47   7.31  


In [9]:
categorical_col = ['Year', 'POS']
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(pred_df[categorical_col]).toarray()

In [10]:
numerical_col = ["Height (in)", "Weight (lbs)", "40 Yard", "Bench Press", "Vert Leap (in)", "Broad Jump (in)", "Shuttle", "3Cone"]
scalar = StandardScaler()
scaled_numerical_features = scalar.fit_transform(pred_df[numerical_col])

In [11]:
pred_data = pd.concat([pd.DataFrame(encoded_features), pd.DataFrame(scaled_numerical_features)], axis=1)
print(pred_data.head())

    0    1    2    3    4    5    6    7    8    9   ...   47   48        0   \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0 -0.354634   
1  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.640600   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.164807   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0 -2.158971   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  0.298726   

         1         2         3         4         5         6         7   
0 -0.458928 -0.012653  0.339983  0.081000 -0.152481  0.247449  0.087713  
1  0.352812  0.235918 -0.012741 -0.320875 -0.503818  0.202952  0.087713  
2  0.533199 -0.154694 -0.012741  0.206586  0.198855  0.291945  0.087713  
3 -1.248120 -1.752653 -1.247275  1.964788  1.721313 -1.487907 -2.463005  
4  0.668489  0.413469 -0.189103 -0.496695 -0.386705  0.336441  0.087713  

[5 rows x 57 columns]


In [12]:
pred_input = pred_data.tail(1)
print(pred_input.to_string())

       0    1    2    3    4    5    6    7    8    9    10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48        0         1         2         3         4         5         6         7 
1840  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.354738  1.119456  0.448979  2.456327  1.085687  0.667304 -0.820462 -1.318452


In [13]:
X_predict = np.array(pred_input)

In [14]:
prediction = model.predict(X_predict)
print("prediction:", prediction)

1/1 [==============================] - 0s 41ms/step
prediction: [[0.436123]]


In [15]:
#2020,WR,73.5,208,4.59,20,31.5,117,4.42,7.31
#2020,WR,73.25,202,4.43,20,37.5,126,4.46,7.31
#2022,RB,72.38,211,4.48,21,30.0,117,4.51,7.31

In [16]:
#2021,WR,71.5,197,4.61,20,38.5,127,4.26,6.9,1
#2021,WR,70.13,180,4.48,20,33.0,122,4.21,6.86,0
#2021,WR,75.0,211,4.49,12,37.0,124,4.43,7.2,0
#2021,WR,72.75,203,4.47,17,40.0,124,4.14,7.02,0